In [39]:
import pandas as pd
import os
import shutil

import warnings
warnings.filterwarnings("ignore")

from deltalake import DeltaTable
from deltalake.writer import write_deltalake

In [40]:
DELTA_LAKE_TABLE = "loans.delta"

In [41]:
if os.path.exists(DELTA_LAKE_TABLE):
    shutil.rmtree(DELTA_LAKE_TABLE, ignore_errors=False, onerror=None)

In [42]:
df = pd.read_csv('./loan.csv')
write_deltalake(DELTA_LAKE_TABLE, df, overwrite_schema=True)

In [43]:
dt = DeltaTable(DELTA_LAKE_TABLE)
print(dt.version())

0


In [44]:
### Fill version=1 in delta lake with selected features from original dataset.

In [45]:
# Remove columns which missing values > 70%
df_1 = df.dropna(axis=1, thresh=int(0.70*len(df)))

print(
    'The number of columns has reduced from {} to {} columns by removing columns with 70% missing values'.
    format(len(df.columns), len(df_1.columns))
)

The number of columns has reduced from 111 to 53 columns by removing columns with 70% missing values


In [46]:
selected_loan_status = ['Fully Paid', 'Charged Off', 'Default']
df_2 = df_1[df_1.loan_status.isin(selected_loan_status)]
df_2.loan_status = df_2.loan_status.replace({'Fully Paid' : 'Good Loan'})
df_2.loan_status = df_2.loan_status.replace({'Charged Off' : 'Bad Loan'})
df_2.loan_status = df_2.loan_status.replace({'Default' : 'Bad Loan'})

print(
    'The number of rows has been reduced from {:,.0f} to {:,.0f} by filtering the data with the correlated loan status'.
    format(len(df_1), len(df_2))     
)

The number of rows has been reduced from 39,717 to 38,577 by filtering the data with the correlated loan status


In [47]:
#### First Trial - Fewer number of features
df_3 = df_2[[
    'loan_status', 'term','int_rate',
    'installment','grade', 'annual_inc',
    'verification_status','dti'  # These features are just initial guess, you can try to choose any other combination
]]
df_3.head()

,loan_status,term,int_rate,installment,grade,annual_inc,verification_status,dti
0,Good Loan,36 months,10.65%,162.87,B,24000.0,Verified,27.65
1,Bad Loan,60 months,15.27%,59.83,C,30000.0,Source Verified,1.00
2,Good Loan,36 months,15.96%,84.33,C,12252.0,Not Verified,8.72
3,Good Loan,36 months,13.49%,339.31,C,49200.0,Source Verified,20.00
5,Good Loan,36 months,7.90%,156.46,A,36000.0,Source Verified,11.20


In [48]:
# Find missing values in the chosen columns
df_null = pd.DataFrame({'Count': df_3.isnull().sum(), 'Percent': round(100*df_3.isnull().sum()/len(df_3),2)})
df_null[df_null['Count'] != 0]

,Count,Percent


In [49]:
# Dropping rows with null values
df_clean = df_3.dropna(axis = 0)

In [50]:
# The next step is to transform categorical target variable into integer
df_clean.loan_status = df_clean.loan_status.replace({'Good Loan' : 1})
df_clean.loan_status = df_clean.loan_status.replace({'Bad Loan' : 0})
df_clean.loan_status.unique()

array([1, 0])

In [51]:
write_deltalake(DELTA_LAKE_TABLE, df_clean, mode='overwrite', overwrite_schema=True)
dt = DeltaTable(DELTA_LAKE_TABLE)
print(dt.version())

1


In [52]:
### Fill version=2 in delta lake with few more features than version1 from original dataset.

In [53]:
df_4 = df_2

In [54]:
# The next step is to transform categorical target variable into integer
df_4.loan_status = df_4.loan_status.replace({'Good Loan' : 1})
df_4.loan_status = df_4.loan_status.replace({'Bad Loan' : 0})

In [55]:
df_4.columns.to_series().groupby(df_clean.dtypes).groups

{int64: ['loan_status'], float64: ['installment', 'annual_inc', 'dti'], object: ['term', 'int_rate', 'grade', 'verification_status']}

In [56]:
# First, dropping categorical features (object type) which have too many options available
df_4 = df_4.drop(['emp_title', 'sub_grade', 'issue_d', 'last_pymnt_d', 'last_credit_pull_d'], axis=1)

In [57]:
# Second, to filter numerical features, we can use .corr() function to select only features with high correlation to the target variable
df_4.corr()['loan_status']

id                           -0.042390
member_id                    -0.039798
loan_amnt                    -0.059463
funded_amnt                  -0.056497
funded_amnt_inv              -0.037781
installment                  -0.027153
annual_inc                    0.040867
loan_status                   1.000000
dti                          -0.045078
delinq_2yrs                  -0.020096
inq_last_6mths               -0.071878
open_acc                      0.009140
pub_rec                      -0.051001
revol_bal                    -0.005854
total_acc                     0.022608
out_prncp                          NaN
out_prncp_inv                      NaN
total_pymnt                   0.235898
total_pymnt_inv               0.232906
total_rec_prncp               0.334944
total_rec_int                -0.013008
total_rec_late_fee           -0.165115
recoveries                   -0.339562
collection_recovery_fee      -0.204914
last_pymnt_amnt               0.222524
collections_12_mths_ex_me

In [58]:
df_clean = df_4[[
    'loan_status', # target variable
    # features (object):
    'term', 'grade','home_ownership', 'verification_status', 'pymnt_plan', 'purpose', 
    'initial_list_status', 'application_type',
    # features (int/float):
    'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'recoveries',                   
    'collection_recovery_fee', 'last_pymnt_amnt', 'int_rate'
]]

In [59]:
df_null = pd.DataFrame({'Count': df_clean.isnull().sum(), 'Percent': round(100*df_clean.isnull().sum()/len(df_clean),2)})
df_null[df_null['Count'] != 0] 

,Count,Percent


In [60]:
write_deltalake(DELTA_LAKE_TABLE, df_clean, mode='overwrite', overwrite_schema=True)
dt = DeltaTable(DELTA_LAKE_TABLE)
print(dt.version())

2
